## Charts

In [1]:
import pandas as pd
import numpy as np

In [2]:
nat_hir = pd.read_csv('natl_home_inc_hir_8918.csv')  # Zillow (summary) + Census annual inc
ca_hir = pd.read_csv('ca_state_home_inc_hir_8918.csv')  # Zillow (summary) + Census annual inc
reg_hir = pd.read_csv('reg_home_inc_hir_8918.csv')  # FHFA home + SAIPE inc (reg_hirs_8918.py)
county_hir = pd.read_csv('county_home_inc_hir_8918.csv')  # FHFA home + SAIPE inc
county_units = pd.read_csv('county_housing_units_8918.csv')  # Census unit counts
hh_count_hir = pd.read_csv('hh_count_hir_8918.csv')
hh_count_per_inc = pd.read_csv('hh_count_housing_per_inc_5yr_0816.csv')
fed = pd.read_csv('fed_int_rate_9020.csv')

nat1 = pd.read_csv('national_rent_vs_mort_and_inc_1yr.csv')
nat5 = pd.read_csv('national_rent_vs_mort_and_inc_5yr.csv')
region1 = pd.read_csv('region_rent_vs_mort_and_inc_1yr.csv')
region5 = pd.read_csv('region_rent_vs_mort_and_inc_5yr.csv')
county1 = pd.read_csv('county_rent_vs_mort_and_inc_1yr.csv')
county5 = pd.read_csv('county_rent_vs_mort_and_inc_5yr.csv')

In [3]:
%run chart_theme.py

---

### National Med. Housing % of Income

In [4]:
# Supplementing 1-yr data w/ 08 & 09 from 5-yr
nat_comb = pd.concat([nat5[nat5.Year < 2010], nat1])

In [5]:
nat_mort_text_df = pd.DataFrame({'Year':(range(2008, 2019)), 'Numbers':[round(i,2) for i in np.arange(0,.55,.05)], 'Text':(['Renters', 'Homeowners'] + list(np.full(9, np.nan)))})

In [159]:
nat_mort = alt.Chart(nat_comb
                    ).mark_line(size=7, strokeCap='round', color='#97c2f2', interpolate='basis'
                               ).encode(x=alt.X('Year:O', axis=alt.Axis(values=[2008,2018], labelFontSize=24, labelOpacity=.7)), 
                                        y=alt.Y('Mort_Share_Inc_dec',
                                               axis=alt.Axis(values=[round(i,2) for i in np.arange(.1,.45,.05)], 
                                                             format='%', 
                                                             labelPadding=40, 
                                                             labelOpacity=0,
                                                             labelFontSize=24), 
                                               scale=alt.Scale(domain=[.05,.4])))

In [160]:
nat_rent = alt.Chart(nat_comb, title='Income Devoted to Housing'
                    ).mark_line(size=7, strokeCap='round', color='#cede54', interpolate='basis'
                               ).encode(x=alt.X('Year:O', axis=alt.Axis(values=[2008,2018], labelFontSize=24, labelOpacity=.7)), 
                                        y=alt.Y('Rent_Share_Inc_dec', title='Housing expenses have trended slightly downwards since 2010',
                                               axis=alt.Axis(values=[round(i,2) for i in np.arange(.1,.45,.05)], 
                                                             format='%', 
                                                             titleX=-40,
                                                             titleY=-40,
                                                             titleOpacity=.7,
                                                             titleFontSize=24,
                                                             labelPadding=40,
                                                             labelOpacity=0,
                                                             labelFontSize=24), 
                                               scale=alt.Scale(domain=[.05,.4])))

renters = alt.Chart(nat_mort_text_df.query('Year == 2008')).mark_text(dx=615, dy=-357, font='lato', fontSize=32, color='#b7c83e').encode(x='Year:O', y='Numbers', text='Text')
homeowners = alt.Chart(nat_mort_text_df.query('Year == 2009')).mark_text(dx=598, dy=-136, font='lato', fontSize=32, color='#81addc').encode(x='Year:O', y='Numbers', text='Text')
y_labels_df = pd.DataFrame({'x':[2007] + [2008]*7, 'y':list(np.arange(.05,.45,.05)), 'text':[str(i) + '    ' for i in range(5,40,5)] + ['40%']})
y_labels = alt.Chart(y_labels_df.query('x == 2008')).mark_text(dx=-40, dy=0, font='lato', fontSize=24, opacity=.7).encode(x='x:O', y='y', text='text')

In [162]:
nat_comb_plot = (nat_mort + nat_rent + renters + homeowners + y_labels).configure_title(fontSize=32)
nat_comb_plot

alt.LayerChart(...)

In [163]:
nat_comb_plot.save('nat_rent_mort_per_inc.png', scale_factor=3)

---

### National Median HIR

In [195]:
natl_med = alt.Chart(nat_hir, title='Home Value Relative to Annual Income'
                    ).mark_line(size=6, strokeCap='round', color='#4c92d8',interpolate='basis'
                               ).encode(x=alt.X('Year:O', axis=alt.Axis(ticks=False, values=list(range(1990, 2019, 4)), 
                                                                        labelOpacity=.7, labelFontSize=18)), 
                                        y=alt.Y('HIR', title='National median home value / median annual household income', 
                                                scale=alt.Scale(domain=[1,4.5]), 
                                                axis=alt.Axis(values=list(np.arange(1.5,4.5,.5)), 
                                                              titleX=-30, titleY=-35, titleOpacity=.7, titleFontSize=20, 
                                                              labelOpacity=.7, labelFontSize=18, labelPadding=30)))

y_text_df = pd.DataFrame({'x':[1989], 'y':[4.5], 'text':['4.5 times income']})
y_text = alt.Chart(y_text_df).mark_text(dx=28, dy=0, fontSize=18, font='lato', opacity=.7).encode(x=alt.X('x:O'), y='y', text='text')
line = alt.Chart(pd.DataFrame({'x1':[1994], 'x2':[2018], 'y1':[4.5]})).mark_rule(opacity=.15).encode(x='x1:O', x2='x2:O', y='y1')

In [196]:
natl_med_plot = (natl_med + y_text + line)
natl_med_plot

alt.LayerChart(...)

In [194]:
natl_med_plot.save('natl_med_hir.png', scale_factor=3)

---

### Regional Med. HIR

Homeowners:

In [164]:
reg_comb = pd.concat([region5[region5.Year < 2010], region1])
reg_mort_text_df = pd.DataFrame({'Year':(range(2008, 2019)), 
                                 'Numbers':[round(i,2) for i in np.arange(0,.55,.05)], 
                                 'Text':(['West', 'Midwest', 'South', 'Northeast'] + list(np.full(7, np.nan)))})

In [196]:
reg_mort = alt.Chart(reg_comb, title='Homeowner Income Devoted to Housing').mark_line(size=5, strokeCap='round', interpolate='basis').encode(x=alt.X('Year:O', axis=alt.Axis(values=list(range(2008, 2019,2)), labelFontSize=24)), 
                                                                                                       y=alt.Y('Mort_Share_Inc_dec', 
                                                                                                               title='By Region', 
                                                                                                               axis=alt.Axis(values=[round(i,2) for i in np.arange(.15,.45,.05)], 
                                                                                                                             format='%', 
                                                                                                                             titleX=-40, 
                                                                                                                             titleOpacity=0,                                                                                                                             
                                                                                                                             labelFontSize=24,                                                                                                                             
                                                                                                                             labelPadding=40,
                                                                                                                             labelOpacity=0), 
                                                                                                               scale=alt.Scale(domain=[.1,.4])), 
                                                                                                       color=alt.Color('Region', 
                                                                                                                       scale=alt.Scale(domain=['Northeast', 'West', 'South', 'Midwest'],
                                                                                                                                       range=['#00a58a', '#004d66', '#ffd5b0', '#c86300'])))

y_labels_df = pd.DataFrame({'x':[2007] + [2008]*6, 'y':list(np.arange(.1,.45,.05)), 'text':[str(i) + '   ' for i in range(10,40,5)] + ['40%']})
y_labels = alt.Chart(y_labels_df.query('x == 2008')).mark_text(dx=-45, dy=0, font='lato', fontSize=24).encode(x='x:O', y='y', text='text')

reg_mort_plot = reg_mort + y_labels
reg_mort_plot = reg_mort_plot.configure_legend(orient='right', titleFont='Lato', titleFontSize=28, titleLimit=250, labelFont='Lato', labelFontSize=28, symbolSize=200, symbolStrokeWidth=5)

alt.LayerChart(...)

In [ ]:
reg_mort_plot

In [197]:
reg_mort_plot.save('reg_homeower_inc_per_housing.png', scale_factor=3)

Renters:

In [192]:
reg_rent = alt.Chart(reg_comb, title='Renter Income Devoted to Housing').mark_line(size=5, strokeCap='round', interpolate='basis').encode(x=alt.X('Year:O', axis=alt.Axis(values=list(range(2008, 2019, 2)), labelFontSize=24)), 
                                                                                                   y=alt.Y('Rent_Share_Inc_dec', 
                                                                                                           title='By region', 
                                                                                                           axis=alt.Axis(values=[round(i,2) for i in np.arange(.15,.45,.05)], 
                                                                                                                         format='%', 
                                                                                                                         titleX=-40, 
                                                                                                                         titleOpacity=0,
                                                                                                                         labelFontSize=24,
                                                                                                                         labelPadding=40, 
                                                                                                                         labelOpacity=0), 
                                                                                                           scale=alt.Scale(domain=[.1,.4])), 
                                                                                                   color=alt.Color('Region', 
                                                                                                                   scale=alt.Scale(domain=['Northeast', 'West', 'South', 'Midwest'], 
                                                                                                                                   range=['#00a58a', '#004d66', '#ffd5b0', '#c86300'])))
y_labels_df = pd.DataFrame({'x':[2007] + [2008]*6, 'y':list(np.arange(.1,.45,.05)), 'text':[str(i) + '   ' for i in range(10,40,5)] + ['40%']})
y_labels = alt.Chart(y_labels_df.query('x == 2008')).mark_text(dx=-45, dy=0, font='lato', fontSize=24).encode(x='x:O', y='y', text='text')

reg_rent_plot = reg_rent + y_labels
reg_rent_plot = reg_rent_plot.configure_legend(orient='right', titleFont='Lato', titleFontSize=28, titleLimit=250, labelFont='Lato', labelFontSize=28, symbolSize=200, symbolStrokeWidth=5)
reg_rent_plot

alt.LayerChart(...)

In [193]:
reg_rent_plot.save('reg_renter_inc_per_housing.png', scale_factor=3)

---

### HIR by Region:

In [151]:
reg_hir['Yr_abbr'] = ['\'' + str(y)[-2:] for y in reg_hir.Year]
region_text_df = pd.DataFrame({'Year':(range(1989, 2019)), 
                               'Numbers':(list(range(30))), 
                               'Text':(['West', 'Northeast', 'South', 'Midwest'] + list(np.full(26, np.nan)))})

In [183]:
reg_chart = alt.Chart(reg_hir, title='Median HIR by Region').mark_line(size=5, strokeCap='round', interpolate='basis').encode(
    x=alt.X('Yr_abbr:O', sort=list(reg_hir.Yr_abbr.unique()), axis=alt.Axis(values=list(reg_hir.Yr_abbr.unique())[1::4], titleFontSize=24, labelFontSize=24)), 
    y=alt.Y('HIR', axis=alt.Axis(titleFontSize=24, labelFontSize=24, values=list(range(2,8)), format='a'), scale=alt.Scale(domain=[1,7.5])),
    color=alt.Color('Region', legend=None, scale=alt.Scale(domain=['West', 'Northeast', 'South', 'Midwest'], range=['#004d66', '#00a58a', '#ffd5b0', '#c86300'])))

west = alt.Chart(region_text_df.query('Numbers == 0')
                   ).mark_text(dx=31, dy=-330, fontSize=28, fontWeight='normal', color='#004d66'
                              ).encode(x=alt.X("Year", axis=None), y=alt.Y("Numbers"), text=alt.Text("Text"))
northeast = alt.Chart(region_text_df.query('Numbers == 1')
                   ).mark_text(dx=60, dy=-199, fontSize=28, fontWeight='normal', color='#00a58a'
                              ).encode(x=alt.X("Year", axis=None), y=alt.Y("Numbers"), text=alt.Text("Text"))
south = alt.Chart(region_text_df.query('Numbers == 2')
                   ).mark_text(dx=35, dy=-85, fontSize=28, fontWeight='normal', color='#ffbe94'
                              ).encode(x=alt.X("Year", axis=None), y=alt.Y("Numbers"), text=alt.Text("Text"))
midwest = alt.Chart(region_text_df.query('Numbers == 3')
                   ).mark_text(dx=51, dy=23, fontSize=28, fontWeight='normal', color='#c86300'
                              ).encode(x=alt.X("Year", axis=None), y=alt.Y("Numbers"), text=alt.Text("Text"))

region_chart = reg_chart + west + northeast + south + midwest
#region_chart = region_chart.configure_title(fontSize=32)
region_chart

alt.LayerChart(...)

In [121]:
region_chart.save('region_hir_8918.png', scale_factor=3)

### CA Counties Med. HIR

Renters:

In [153]:
# Relied on 5-yr for 08-16 and supplemented 1-yr for 17 & 18 since county data is succeptible to greater change between years (due to smaller sample data pools)
ca0818 = pd.concat([county5[(county5.FIPS > 6000) & (county5.FIPS < 7000) & (county5.FIPS.isin(county1.FIPS.unique()))], 
                    county1[(county1.Year > 2016) & (county1.FIPS > 6000) & (county1.FIPS < 7000)]])
la = ca0818[ca0818.FIPS == 6037]
ca_no_la = ca0818[ca0818.FIPS != 6037]

In [156]:
ca_rent_per = alt.Chart(ca_no_la, title='Renter Income Devoted to Housing'
                       ).mark_line(opacity=.6, interpolate='basis'
                                  ).encode(x=alt.X('Year:O', axis=alt.Axis(labelFontSize=20, values=list(range(2008,2020,2)))), 
                                           y=alt.Y('Rent_Share_Inc_dec', 
                                                    title='LA relative to all other CA counties', 
                                                    axis=alt.Axis(format='%', 
                                                                  values=[round(i,2) for i in np.arange(.15,.55,.05)], 
                                                                  titleFontSize=24, titleX=-40, titleY=-30, 
                                                                  labelFontSize=20, labelPadding=40, labelOpacity=0), 
                                                    scale=alt.Scale(domain=[.1,.5])), 
                                            color=alt.Color('FIPS:O', scale=alt.Scale(range=['#e0e0e5']), legend=None))
la_rent_per = alt.Chart(la).mark_line(color='#fbcb42', size=6, strokeCap='round', interpolate='basis').encode(x=alt.X('Year:O', axis=alt.Axis(labelFontSize=18, values=list(range(2008,2020,2)))), 
                                                                      y=alt.Y('Rent_Share_Inc_dec', 
                                                                              axis=alt.Axis(values=[round(i,2) for i in np.arange(.15,.55,.05)], format='%'), 
                                                                              scale=alt.Scale(domain=[.1,.5])))
y_labels_df = pd.DataFrame({'x':[2007] + [2008]*8, 'y':list(np.arange(.1,.55,.05)), 'text':[str(i) + '   ' for i in range(10,50,5)] + ['50%']})
y_labels = alt.Chart(y_labels_df.query('x == 2008')).mark_text(dx=-45, dy=0, font='lato', fontSize=20).encode(x='x:O', y='y', text='text')

rent_per_inc = ca_rent_per + la_rent_per + y_labels

In [157]:
rent_per_inc

alt.LayerChart(...)

In [160]:
rent_per_inc.save('renter_inc_per_housing.png', scale_factor=3)

Homeowners:

In [158]:
ca_mort_per = alt.Chart(ca_no_la, title='Homeowner Income Devoted to Housing'
                       ).mark_line(opacity=.6, interpolate='basis'
                                  ).encode(x=alt.X('Year:O', 
                                                   axis=alt.Axis(labelFontSize=20, 
                                                                 values=list(range(2008,2020,2)))), 
                                           y=alt.Y('Mort_Share_Inc_dec', 
                                                    title='LA relative to all other CA counties', 
                                                    axis=alt.Axis(values=[round(i,2) for i in np.arange(.15,.55,.05)], 
                                                                  titleFontSize=24, titleX=-40, titleY=-30, 
                                                                  labelPadding=40, labelFontSize=20, labelOpacity=0, format='%'), 
                                                    scale=alt.Scale(domain=[.1,.5])), 
                                           color=alt.Color('FIPS:O', scale=alt.Scale(range=['#e0e0e5']), legend=None))
la_mort_per = alt.Chart(la).mark_line(color='#fbcb42', size=6, strokeCap='round', interpolate='basis').encode(x=alt.X('Year:O', axis=alt.Axis(labelFontSize=18, values=list(range(2008,2020,2)))), 
                                                                                         y=alt.Y('Mort_Share_Inc_dec', 
                                                                                                 axis=alt.Axis(values=[round(i,2) for i in np.arange(.15,.55,.05)], format='%'), 
                                                                                                 scale=alt.Scale(domain=[.1,.5])))
y_labels_df = pd.DataFrame({'x':[2007] + [2008]*8, 'y':list(np.arange(.1,.55,.05)), 'text':[str(i) + '   ' for i in range(10,50,5)] + ['50%']})
y_labels = alt.Chart(y_labels_df.query('x == 2008')).mark_text(dx=-45, dy=0, font='lato', fontSize=20).encode(x='x:O', y='y', text='text')

mort_per_inc = ca_mort_per + la_mort_per + y_labels

In [159]:
mort_per_inc

alt.LayerChart(...)

In [161]:
mort_per_inc.save('homeowner_inc_per_housing.png', scale_factor=3)

In [163]:
ca0818[['Year', 'County', 'Mort_Share_Inc_whole']][ca0818.Year == 2016].sort_values(['Year','Mort_Share_Inc_whole'], ascending=[True, False]).head(10)

,Year,County,Mort_Share_Inc_whole
25186,2016,"Los Angeles County, California",24.6
25200,2016,"Riverside County, California",23.9
25204,2016,"San Diego County, California",23.8
25211,2016,"Santa Cruz County, California",23.6
25223,2016,"Ventura County, California",23.6
25188,2016,"Marin County, California",23.2
25196,2016,"Nevada County, California",23.2
25197,2016,"Orange County, California",23.1
25207,2016,"San Luis Obispo County, California",22.7
25216,2016,"Sonoma County, California",22.5


---

### HIR County Distribution

2018 Distribution:

In [5]:
plot_df = county_hir.copy()
plot_df['pct_bin'] = plot_df.HIR.round(1)

In [7]:
yrs_dict = {}
for year in plot_df.Year.unique():
    yr_dict = {}    
    yr_df = plot_df[plot_df.Year == year]
    for val in yr_df.pct_bin.unique():
        yr_dict[val] = len(yr_df[yr_df.pct_bin == val])
    yrs_dict[year] = yr_dict

In [19]:
# Identifying max to manipulate chart so y-axis is consistent for all charts
for year in yrs_dict:
    print(year, max(yrs_dict[year].values()))

1989 102
1990 109
1991 116
1992 134
1993 161
1994 155
1995 164
1996 165
1997 167
1998 169
1999 193
2000 166
2001 171
2002 174
2003 153
2004 150
2005 150
2006 133
2007 133
2008 139
2009 152
2010 188
2011 176
2012 186
2013 195
2014 204
2015 191
2016 194
2017 196
2018 180


In [20]:
# Identifying max to manipulate chart so y-axis is consistent for all charts
for year in yrs_dict:
    print(year, max(yrs_dict[year].keys()))

1989 14.3
1990 12.8
1991 11.6
1992 10.4
1993 9.4
1994 8.3
1995 8.2
1996 7.9
1997 7.8
1998 8.9
1999 9.2
2000 10.6
2001 11.4
2002 11.7
2003 13.0
2004 13.5
2005 15.6
2006 17.6
2007 17.8
2008 15.4
2009 15.0
2010 13.7
2011 13.0
2012 13.0
2013 13.0
2014 12.4
2015 13.1
2016 15.2
2017 13.6
2018 13.4


In [492]:
counts_df = pd.DataFrame(yrs_dict[2018].items(), columns=['Bins', 'Counts']).sort_values(['Bins'])
counts_df = counts_df[(counts_df.Bins < 7) & ~((counts_df.Bins > 6) & (counts_df.Counts > 3))]
text_line_df = pd.DataFrame({'x1':[2.75], 'x2':[5.2], 'x3':[5.7], 'x4':[5.6], 'x5':[5.78], 'y1':[179], 'y2':[179], 'y3':[160], 'y4':[150], 'y5':[140], 'Text1':['In 180 counties, median home value'], 'Text2':['was 2.6 times median household income']})
text_df = pd.DataFrame({'Bins':[1,7], 'Counts':[200,200], 'Text':[1989, 2018]})

# Charts
chart = alt.Chart(counts_df, title='Home Value Relative to Annual Income (2018)'
                 ).mark_bar(size=7.5, color='#4c92d8', cornerRadiusTopLeft=1, cornerRadiusTopRight=1
                 ).encode(x=alt.X('Bins', title='Home/Income Multiple',
                                  axis=alt.Axis(values=list(range(8)), titleY=30, titleFontSize=20, titleOpacity=.7, labelOpacity=0)), 
                          y=alt.Y('Counts:Q', title='Distribution of median values for all counties',
                                  axis=alt.Axis(values=list(range(40,200,40)), titleX=-30, titleY=0, titleFontSize=20, titleOpacity=.7,
                                                labelPadding=30, labelFontSize=18, labelOpacity=.7), 
                                                scale=alt.Scale(domain=[0,210])))
x_vals = alt.Chart(pd.DataFrame({'x':list(range(1,8)), 'y':[0]*7, 'text':list(range(1,8))})
                  ).mark_text(dx=0, dy=20, fontSize=18, opacity=.7
                             ).encode(x=alt.X('x'), y=alt.Y('y', axis=None), text='text')

y_text = alt.Chart(pd.DataFrame({'x':[1], 'y':[200], 'text':['200 counties']})).mark_text(dx=-21, dy=0, fontSize=18, font='lato', opacity=.7).encode(x='x', y='y', text='text')
line = alt.Chart(pd.DataFrame({'x1':[1.32], 'x2':[7], 'y1':[200]})).mark_rule(opacity=.15).encode(x='x1', x2='x2', y='y1')
line1 = alt.Chart(text_line_df).mark_rule(size=2, color='#999999', strokeDash=[1,2.5]).encode(x='x1', x2='x2', y='y1', y2='y2')
line2 = alt.Chart(text_line_df).mark_rule(size=2, color='#999999', strokeDash=[1,2.5]).encode(x='x2', x2='x3', y='y2', y2='y3')
text1 = alt.Chart(text_line_df.query('x1 == 2.75')).mark_text(dx=0, dy=0, fontSize=16, font='lato', opacity=.7).encode(x='x4', y='y4', text='Text1')
text2 = alt.Chart(text_line_df.query('x1 == 2.75')).mark_text(dx=0, dy=0, fontSize=16, font='lato', opacity=.7).encode(x='x5', y='y5', text='Text2')

plot = chart + x_vals + y_text + line + line1 + line2 + text1 + text2

In [438]:
plot

alt.LayerChart(...)

In [410]:
plot.save('hir_county_dist_2018.png', scale_factor=3)

---

#### 1989-1999 Distribution

In [64]:
years = range(1989,2000)
variables = []
counter = 0
for year in years:
    df = pd.DataFrame(yrs_dict[year].items(), columns=['Bins', 'Counts']).sort_values(['Bins'])
    df = df[df.Bins <= 7]  # Removing outliers (to ensure consistent x axis values)
    add_vals = []
    for val in np.arange(0,7,.1):
        if round(val,1) not in df.Bins.values:
            add_vals.append(round(val,1))
    missing_vals = pd.DataFrame({'Bins':add_vals, 'Counts':list(np.full(len(add_vals),0))})
    df = pd.concat([df, missing_vals]).sort_values('Bins')
    
    var = alt.Chart(df, title='Home Value Relative to Annual Income (\'89-\'99)'
                   ).mark_bar(color='#4c92d8', size=8, opacity=.3, cornerRadiusTopLeft=1, cornerRadiusTopRight=1
                             ).encode(x=alt.X('Bins:O', title='Home/Income Multiple', 
                                              axis=alt.Axis(values=list(range(1,8)), 
                                                            titleOpacity=.7, titleFontSize=20, titleY=35, 
                                                            labelFontSize=18, labelOpacity=.7)), 
                                      y=alt.Y('Counts:Q', scale=alt.Scale(domain=[0,210]), 
                                              axis=alt.Axis(titleFontSize=20, titleX=-35, 
                                                            labelOpacity=.7, labelPadding=35, labelFontSize=18, 
                                                            values=list(range(40,200,40)))))
    y_text = alt.Chart(pd.DataFrame({'x':[1], 'y':[200], 'text':['200 counties']})).mark_text(dx=-72, dy=0, fontSize=18, font='lato', fontWeight='normal', opacity=.09).encode(x='x:O', y='y', text='text')
    line = alt.Chart(pd.DataFrame({'x1':[.8], 'x2':[7], 'y1':[200]})).mark_rule(opacity=.01).encode(x='x1:O', x2='x2', y='y1')
    chart = var + y_text + line
    variables.append(chart)
    counter += 1

combined8999 = chart   # Last chart created (must assign altair chart object to variable to enable appending)
for var in variables[:-1]:  # Excluding last chart (b/c it's already assigned to 'test')
    combined8999 += var

combined8999

alt.LayerChart(...)

In [65]:
combined8999.save('combined8999.png', scale_factor=3)

---

#### 2000-2010 Distribution

In [79]:
years = range(2000,2010)
variables = []
counter = 0
for year in years:
    df = pd.DataFrame(yrs_dict[year].items(), columns=['Bins', 'Counts']).sort_values(['Bins'])
    df = df[df.Bins <= 7]  # Removing outliers (to ensure consistent x axis values)
    add_vals = []
    for val in np.arange(0,7,.1):
        if round(val,1) not in df.Bins.values:
            add_vals.append(round(val,1))
    missing_vals = pd.DataFrame({'Bins':add_vals, 'Counts':list(np.full(len(add_vals),0))})
    df = pd.concat([df, missing_vals]).sort_values('Bins')
    
    var = alt.Chart(df, title='Home Value Relative to Annual Income (\'00-\'09)'
                   ).mark_bar(color='#4c92d8', size=8, opacity=.3, cornerRadiusTopLeft=1, cornerRadiusTopRight=1
                             ).encode(x=alt.X('Bins:O', title='Home/Income Multiple', axis=alt.Axis(values=list(range(1,8)), titleFontSize=20, titleY=35, labelFontSize=18, labelOpacity=.7)), 
                                      y=alt.Y('Counts:Q', scale=alt.Scale(domain=[0,210]), 
                                              axis=alt.Axis(titleFontSize=20, titleX=-35, 
                                                            labelPadding=35, labelFontSize=18, labelOpacity=.7, 
                                                            values=list(range(40,200,40)))))
    y_text = alt.Chart(pd.DataFrame({'x':[1], 'y':[200], 'text':['200 counties']})).mark_text(dx=-72, dy=0, fontSize=18, font='lato', fontWeight='normal', opacity=.1).encode(x='x:O', y='y', text='text')
    line = alt.Chart(pd.DataFrame({'x1':[.8], 'x2':[7], 'y1':[200]})).mark_rule(opacity=.01).encode(x='x1:O', x2='x2', y='y1')
    chart = var + y_text + line
    variables.append(chart)
    counter += 1

combined0009 = var   # Last chart created (must assign altair chart object to variable to enable appending)
for var in variables[:-1]:  # Excluding last chart
    combined0009 += var

combined0009

alt.LayerChart(...)

In [80]:
combined0009.save('combined0009.png', scale_factor=3)

---

#### 2010-2018 Distribution

In [81]:
years = range(2010,2019)
variables = []
counter = 0
for year in years:
    df = pd.DataFrame(yrs_dict[year].items(), columns=['Bins', 'Counts']).sort_values(['Bins'])
    df = df[df.Bins <= 7]  # Removing outliers (to ensure consistent x axis values)
    add_vals = []
    for val in np.arange(0,7,.1):
        if round(val,1) not in df.Bins.values:
            add_vals.append(round(val,1))
    missing_vals = pd.DataFrame({'Bins':add_vals, 'Counts':list(np.full(len(add_vals),0))})
    df = pd.concat([df, missing_vals]).sort_values('Bins')
    
    var = alt.Chart(df, title='Home Value Relative to Annual Income (\'10-\'18)'
                   ).mark_bar(color='#4c92d8', size=8, opacity=.3, cornerRadiusTopLeft=1, cornerRadiusTopRight=1
                             ).encode(x=alt.X('Bins:O', title='Home/Income Multiple', axis=alt.Axis(values=list(range(1,8)), titleFontSize=20, labelFontSize=18, labelOpacity=.7, titleY=35)), 
                                      y=alt.Y('Counts:Q',
                                              scale=alt.Scale(domain=[0,210]), 
                                              axis=alt.Axis(titleFontSize=20, titleX=-35, labelPadding=35, labelFontSize=18, labelOpacity=.7, values=list(range(40,200,40)))))
    y_text = alt.Chart(pd.DataFrame({'x':[1], 'y':[200], 'text':['200 counties']})).mark_text(dx=-72, dy=0, fontSize=18, font='lato', fontWeight='normal', opacity=.12).encode(x='x:O', y='y', text='text')
    line = alt.Chart(pd.DataFrame({'x1':[.8], 'x2':[7], 'y1':[200]})).mark_rule(opacity=.02).encode(x='x1:O', x2='x2', y='y1')
    chart = var + y_text + line
    variables.append(chart)    
    counter += 1

combined1018 = var   # Last chart created (must assign altair chart object to variable to enable appending)
for var in variables[:-1]:  # Excluding last chart
    combined1018 += var

combined1018

alt.LayerChart(...)

In [82]:
combined1018.save('combined1018.png', scale_factor=3)

---

### HIR by County Household Count:

In [84]:
hir_pop = alt.Chart(hh_count_hir, title='HIR by County Household Count').mark_line(size=7, strokeCap='round', interpolate='basis').encode(
x=alt.X('Year:O', axis=alt.Axis(values=list(range(1990, 2019,4)), labelFontSize=24, labelOpacity=.7)),
y=alt.Y('HIR', title='Median HIR for county groupings by number of households', scale=alt.Scale(domain=[0,8]), axis=alt.Axis(titleY=-30, titleFontSize=24, labelFontSize=24, labelOpacity=.7, values=list(range(1,9)), format='a')),
color=alt.Color('HH_Count', scale=alt.Scale(domain=list(hh_count_hir.HH_Count.unique()), 
                                             range=['#b2182b','#ef8a62','#fddbc7','#f8f6e9','#d1e5f0','#67a9cf','#2166ac']), title='Households (thousands)')
).configure_title(fontSize=32).configure_legend(orient='none', legendX=610, legendY=87, titleFont='Lato', titleFontSize=20, titleLimit=250, labelFont='Lato', labelFontSize=20, symbolSize=300, symbolStrokeWidth=7)

hir_pop

alt.Chart(...)

In [43]:
hir_pop.save('hir_hh_count.png', scale_factor=3)

---

### Housing % of Income (by county size)

In [7]:
# Manipulating data to plot multiple lines on chart
rent = hh_count_per_inc[hh_count_per_inc.Type == 'Rents']
mort = hh_count_per_inc[hh_count_per_inc.Type == 'Morts']
df_dict = {}
for type_df in [rent, mort]:
    vals = []
    for col in type_df.columns[:-2]:
        vals += type_df[col].values.tolist()
    df = pd.DataFrame({'Year':list(range(2008, 2017))*7, 'Home_Per_Inc':[round(i,3) for i in vals]})
    house_types = []
    for typ in type_df.columns[:-2]:
        house_types += [typ]*9
    df['Type'] = house_types
    df_dict[type_df.Type.iloc[0]] = df

In [33]:
hh_count_mort_df = pd.DataFrame(df_dict['Morts'])
hh_count_mort = alt.Chart(hh_count_mort_df, title='Homeowner Income Devoted to Housing'
                    ).mark_line(size=7, strokeCap='round', interpolate='basis'
                               ).encode(x=alt.X('Year:O', axis=alt.Axis(values=list(range(2008,2018,2)), labelFontSize=24)), 
                                        y=alt.Y('Home_Per_Inc', title='Counties grouped by number of households',
                                               axis=alt.Axis(values=[round(i,2) for i in np.arange(.1,.45,.05)], 
                                                             format='%', 
                                                             titleX=-35,
                                                             titleY=-35,
                                                             titleFontSize=24,
                                                             labelPadding=35, 
                                                             labelOpacity=0,
                                                             labelFontSize=24), 
                                               scale=alt.Scale(domain=[.05,.4])),
                                        color=alt.Color('Type', scale=alt.Scale(domain=list(hh_count_mort_df.Type.unique()), 
                                             range=['#b2182b','#ef8a62','#fddbc7','#f8f6e9','#d1e5f0','#67a9cf','#2166ac']), title='Households (thousands)'))

y_labels_df = pd.DataFrame({'x':[2007] + [2008]*7, 'y':list(np.arange(.05,.45,.05)), 'text':[str(i) + '   ' for i in range(5,40,5)] + ['40%']})
y_labels = alt.Chart(y_labels_df.query('x == 2008')).mark_text(dx=-45, dy=0, font='lato', fontSize=24).encode(x='x:O', y='y', text='text')


hh_count_mort_chart = hh_count_mort + y_labels
hh_count_mort_chart = hh_count_mort_chart.configure_title(fontSize=32).configure_legend(titleFont='Lato', titleFontSize=20, titleLimit=250, labelFont='Lato', labelFontSize=20, symbolSize=300, symbolStrokeWidth=7)
hh_count_mort_chart

alt.LayerChart(...)

In [34]:
hh_count_rent_df = pd.DataFrame(df_dict['Rents'])
hh_count_rent = alt.Chart(hh_count_rent_df, title='Renter Income Devoted to Housing'
                    ).mark_line(size=7, strokeCap='round', interpolate='basis'
                               ).encode(x=alt.X('Year:O', axis=alt.Axis(values=list(range(2008,2018,2)), labelFontSize=24)), 
                                        y=alt.Y('Home_Per_Inc', title='Counties grouped by number of households',
                                               axis=alt.Axis(values=[round(i,2) for i in np.arange(.1,.45,.05)], 
                                                             format='%', 
                                                             titleX=-35,
                                                             titleY=-35,
                                                             titleFontSize=24,
                                                             labelPadding=35, 
                                                             labelOpacity=0,
                                                             labelFontSize=24), 
                                               scale=alt.Scale(domain=[.05,.4])),
                                        color=alt.Color('Type', scale=alt.Scale(domain=list(hh_count_rent_df.Type.unique()), 
                                             range=['#b2182b','#ef8a62','#fddbc7','#f8f6e9','#d1e5f0','#67a9cf','#2166ac']), title='Households (thousands)'))

y_labels_df = pd.DataFrame({'x':[2007] + [2008]*7, 'y':list(np.arange(.05,.45,.05)), 'text':[str(i) + '   ' for i in range(5,40,5)] + ['40%']})
y_labels = alt.Chart(y_labels_df.query('x == 2008')).mark_text(dx=-45, dy=0, font='lato', fontSize=24).encode(x='x:O', y='y', text='text')


hh_count_rent_chart = hh_count_rent + y_labels
hh_count_rent_chart = hh_count_rent_chart.configure_title(fontSize=32).configure_legend(titleFont='Lato', titleFontSize=20, titleLimit=250, labelFont='Lato', labelFontSize=20, symbolSize=300, symbolStrokeWidth=7)
hh_count_rent_chart

alt.LayerChart(...)

In [37]:
hh_count_mort_chart.save('hh_count_mort_chart.png', scale_factor=3)

In [38]:
hh_count_rent_chart.save('hh_count_rent_chart.png', scale_factor=3)

---

### LA vs. Other HIR 

In [91]:
# Ranking CA Counties by Avg. HIR
ca_counties = county_hir[(county_hir.FIPS > 6000) & (county_hir.FIPS < 7000)]
fips_dict = {}
for fips in ca_counties.FIPS.unique():
    df = ca_counties[ca_counties.FIPS == fips]
    fips_dict[fips] = df.HIR.mean().round(3)
    
avg_hirs = sorted(fips_dict.items(), key=lambda x:x[1], reverse=True)

In [92]:
avg_hirs[:8]  # LA has the 6th highest avg. HIR in CA

[(6075, 9.089),
 (6087, 8.465),
 (6041, 8.078),
 (6081, 7.745),
 (6083, 7.408),
 (6037, 7.283),
 (6079, 7.277),
 (6045, 7.181)]

In [93]:
la_df = county_hir[county_hir.FIPS == 6037]

In [94]:
ca_df = county_hir[(county_hir.FIPS != 6037) & (county_hir.FIPS > 6000) & (county_hir.FIPS < 7000)]

In [95]:
text_df = la_df.copy()
text_df['Text'] = list(np.full(len(text_df)-5, np.nan)) + ['CA', 'Counties', 'National', 'California', 'LA County']

In [97]:
ca_counties = alt.Chart(ca_df, title='LA vs. Other Regions').mark_line(opacity=.6, interpolate='basis').encode(
    x=alt.X('Year:O', axis=alt.Axis(values=list(range(1990, 2019,4)), labelFontSize=18)), 
    y=alt.Y('HIR', title='LA county median HIR relative to CA median, nat\'l median and CA\'s other counties', scale=alt.Scale(domain=[0,14]), axis=alt.Axis(values=list(range(2,16,2)), titleFontSize=18, labelFontSize=18)),
    color=alt.Color('FIPS:O', scale=alt.Scale(range=['#e0e0e5']), legend=None))

nat_med = alt.Chart(nat_hir).mark_line(color='#9a9a9a', size=5, strokeCap='round', interpolate='basis').encode(
    x=alt.X('Year:O'), 
    y=alt.Y('HIR'))

ca_med = alt.Chart(ca_hir).mark_line(size=5, strokeCap='round', interpolate='basis').encode(
    x=alt.X('Year:O'), 
    y=alt.Y('HIR'))

la = alt.Chart(la_df).mark_line(color='#fbcb42', size=5, strokeCap='round', interpolate='basis').encode(
    x=alt.X('Year:O'), 
    y=alt.Y('HIR'))

CA_text = (
    alt.Chart(text_df.query('Year == 2014'))
    .mark_text(dx=103, dy=60, fontSize=20, fontWeight='normal', color='#afafaf')
    .encode(x=alt.X("Year:O", axis=None), y=alt.Y("HIR"), text=alt.Text("Text"))
)

cou_text = (
    alt.Chart(text_df.query('Year == 2015'))
    .mark_text(dx=109, dy=83, fontSize=20, fontWeight='normal', color='#afafaf')
    .encode(x=alt.X("Year:O", axis=None), y=alt.Y("HIR"), text=alt.Text("Text"))
)
nat_text = (
    alt.Chart(text_df.query('Year == 2016'))
    .mark_text(dx=94, dy=141, fontSize=22, fontWeight='bold', color='#858585')
    .encode(x=alt.X("Year:O", axis=None), y=alt.Y("HIR"), text=alt.Text("Text"))
)

ca_text = (
    alt.Chart(text_df.query('Year == 2017'))
    .mark_text(dx=80, dy=28, fontSize=22, fontWeight='bold', color='#15607a')
    .encode(x=alt.X("Year:O", axis=None), y=alt.Y("HIR"), text=alt.Text("Text"))
)

la_text = (
    alt.Chart(text_df.query('Year == 2018'))
    .mark_text(dx=65, dy=-3, fontSize=22, fontWeight='bold', color='#ffca00')
    .encode(x=alt.X("Year:O", axis=None), y=alt.Y("HIR"), text=alt.Text("Text"))
)

chart = ca_counties + nat_med + ca_med + la + CA_text + cou_text + nat_text + ca_text + la_text
chart = chart.configure_title(fontSize=28)
chart

alt.LayerChart(...)

In [98]:
chart.save('la_ca_nat_counties_hir.png', scale_factor=3)

---

## Fed Funds Rate

In [6]:
fed_plot = alt.Chart(fed, title='Target Fed Funds Rate').mark_line(size=2.5).encode(x=alt.X('Date:T', axis=alt.Axis(labelOpacity=.7, labelFontSize=18)), 
                                                                         y=alt.Y('Target_Rate', 
                                                                                 title='Benchmark interest rate set by the Federal Reserve', 
                                                                                 axis=alt.Axis(values=list(np.arange(.01,.09,.01)), 
                                                                                               titleX=-30, titleY=-35, titleOpacity=.7, titleFontSize=20, 
                                                                                               labelOpacity=.7, labelFontSize=18, labelPadding=30, format='%'))).properties(width=600)
fed_plot

alt.Chart(...)

In [7]:
fed_plot.save('fed_funds_rate.png', scale_factor=3)

---